##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [30]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Better performance with tf.function and AutoGraph

TF 2.0 brings together the ease of eager execution and the power of TF 1.0. At the center of this merger is `tf.function`, which allows you to transform a subset of Python syntax into portable, high-performance TensorFlow graphs.

A cool new feature of `tf.function` is AutoGraph, which lets you write graph code using natural Python syntax. For a list of the Python features that you can use with AutoGraph, see [AutoGraph Capabilities and Limitations](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md). For more details about `tf.function`, see the RFC [TF 2.0: Functions, not Sessions](https://github.com/tensorflow/community/blob/master/rfcs/20180918-functions-not-sessions-20.md). For more details about AutoGraph, see `tf.autograph`.

This tutorial will walk you through the basic features of `tf.function` and AutoGraph.

## Setup

Import TensorFlow 2.0:

In [31]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np

In [32]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

## The `tf.function` decorator

When you annotate a function with `tf.function`, you can still call it like any other function. But it will be compiled into a graph, which means you get the benefits of faster execution, running on GPU or TPU, or exporting to SavedModel.

In [33]:
@tf.function
def simple_nn_layer(x, y):
  return tf.nn.relu(tf.matmul(x, y))


x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

simple_nn_layer(x, y)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0.25601318, 0.38919708, 0.34901285],
       [0.14291635, 0.3470501 , 0.2846484 ],
       [0.16643555, 0.366066  , 0.5762576 ]], dtype=float32)>

If we examine the result of the annotation, we can see that it's a special callable that handles all interactions with the TensorFlow runtime.

In [34]:
simple_nn_layer

If your code uses multiple functions, you don't need to annotate them all - any functions called from an annotated function will also run in graph mode.

In [35]:
def linear_layer(x):
  return 2 * x + 1


@tf.function
def deep_net(x):
  return tf.nn.relu(linear_layer(x))


deep_net(tf.constant((1, 2, 3)))

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([3, 5, 7])>

Functions can be faster than eager code, for graphs with many small ops. But for graphs with a few expensive ops (like convolutions), you may not see much speedup.


In [36]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])
# warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))
print("Note how there's not much difference in performance for convolutions")


Eager conv: 0.7773740999982692
Function conv: 0.6933108999946853
Note how there's not much difference in performance for convolutions


In [38]:
print(conv_layer)
print(conv_fn)

In [39]:
lstm_cell = tf.keras.layers.LSTMCell(10)

@tf.function
def lstm_fn(input, state):
  return lstm_cell(input, state)

input = tf.zeros([10, 10])
state = [tf.zeros([10, 10])] * 2
# warm up
lstm_cell(input, state); lstm_fn(input, state)
print("eager lstm:", timeit.timeit(lambda: lstm_cell(input, state), number=10))
print("function lstm:", timeit.timeit(lambda: lstm_fn(input, state), number=10))


eager lstm: 0.022824300001957454
function lstm: 0.01508789999934379


## Use Python control flow

When using data-dependent control flow inside `tf.function`, you can use Python control flow statements and AutoGraph will convert them into appropriate TensorFlow ops. For example, `if` statements will be converted into `tf.cond()` if they depend on a `Tensor`.

In the example below, `x` is a `Tensor` but the `if` statement works as expected:

In [41]:
@tf.function
def square_if_positive(x):
  if x > 0:
    x = x * x
  else:
    x = 0
  return x

print(square_if_positive(tf.constant(2)))
print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

tf.Tensor(4, shape=(), dtype=int32)
square_if_positive(2) = 4
square_if_positive(-2) = 0


Note: The previous example uses simple conditionals with scalar values. <a href="#batching">Batching</a> is typically used in real-world code.

AutoGraph supports common Python statements like `while`, `for`, `if`, `break`, `continue` and `return`, with support for nesting. That means you can use `Tensor` expressions in the condition of `while` and `if` statements, or iterate over a `Tensor` in a `for` loop.

In [10]:
@tf.function
def sum_even(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s


sum_even(tf.constant([10, 12, 15, 20]))

<tf.Tensor: shape=(), dtype=int32, numpy=42>

AutoGraph also provides a low-level API for advanced users. For example we can use it to have a look at the generated code.

In [11]:
print(tf.autograph.to_code(sum_even.python_function))

def tf__sum_even(items):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('sum_even', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
    s = 0

    def get_state_2():
      return ()

    def set_state_2(_):
      pass

    def loop_body(iterates, s):
      c = iterates
      continue_ = False

      def get_state():
        return ()

      def set_state(_):
        pass

      def if_true():
        continue_ = True
        return continue_

      def if_false():
        return continue_
      cond = c % 2 > 0
      continue_ = ag__.if_stmt(cond, if_true, if_false, get_state, set_state, ('continue_',), ())

      def get_state_1():
        return ()

      def set_state_1(_):
        pass

      def if_true_1():
        s_1, = s,
        s_1 += c
        return s_1

      def if_false_1():
        return s
      cond_1 = ag__.not_(continue_)
      s = ag__.if

Here's an example of more complicated control flow:

In [12]:
@tf.function
def fizzbuzz(n):
  for i in tf.range(n):
    if i % 3 == 0:
      tf.print('Fizz')
    elif i % 5 == 0:
      tf.print('Buzz')
    else:
      tf.print(i)

fizzbuzz(tf.constant(15))

Fizz
1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14


## Keras and AutoGraph

AutoGraph is available by default in non-dynamic Keras models. For more information, see `tf.keras`.

In [13]:
class CustomModel(tf.keras.models.Model):

  @tf.function
  def call(self, input_data):
    if tf.reduce_mean(input_data) > 0:
      return input_data
    else:
      return input_data // 2


model = CustomModel()

model(tf.constant([-2, -4]))

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([-1, -2])>

## Side effects

Just like in eager mode, you can use operations with side effects, like `tf.assign` or `tf.print` normally inside `tf.function`, and it will insert the necessary control dependencies to ensure they execute in order.

In [14]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
  v.assign(v + 1)
  if v % 2 == 0:
    v.assign(v + 1)


find_next_odd()
v

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=7>

<a id="debugging"></a>

## Debugging

`tf.function` and AutoGraph work by generating code and tracing it into TensorFlow graphs. This mechanism does not yet support step-by-step debuggers like `pdb`. However, you can call `tf.config.run_functions_eagerly(True)` to temporarily enable eager execution inside the `tf.function' and use your favorite debugger:

In [15]:
@tf.function
def f(x):
  if x > 0:
    # Try setting a breakpoint here!
    # Example:
    #   import pdb
    #   pdb.set_trace()
    x = x + 1
  return x

tf.config.experimental_run_functions_eagerly(True)

# You can now set breakpoints and run the code in a debugger.
f(tf.constant(1))

tf.config.experimental_run_functions_eagerly(False)

## Advanced example: An in-graph training loop

The previous section showed that AutoGraph can be used inside Keras layers and models. Keras models can also be used in AutoGraph code.

This example shows how to train a simple Keras model on MNIST with the entire training process—loading batches, calculating gradients, updating parameters, calculating validation accuracy, and repeating until convergence—is performed in-graph.

### Download data

In [16]:
def prepare_mnist_features_and_labels(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  y = tf.cast(y, tf.int64)
  return x, y

def mnist_dataset():
  (x, y), _ = tf.keras.datasets.mnist.load_data()
  ds = tf.data.Dataset.from_tensor_slices((x, y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.take(20000).shuffle(20000).batch(100)
  return ds

train_dataset = mnist_dataset()

### Define the model

In [17]:
model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))
model.build()
optimizer = tf.keras.optimizers.Adam()

### Define the training loop

In [18]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


def train_one_step(model, optimizer, x, y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(y, logits)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  compute_accuracy(y, logits)
  return loss


@tf.function
def train(model, optimizer):
  train_ds = mnist_dataset()
  step = 0
  loss = 0.0
  accuracy = 0.0
  for x, y in train_ds:
    step += 1
    loss = train_one_step(model, optimizer, x, y)
    if step % 10 == 0:
      tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
  return step, loss, accuracy

step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())

Step 10 : loss 1.89959216 ; accuracy 0.356
Step 20 : loss 1.25947154 ; accuracy 0.4975
Step 30 : loss 0.874856472 ; accuracy 0.587
Step 40 : loss 0.661576748 ; accuracy 0.64475
Step 50 : loss 0.45435676 ; accuracy 0.689
Step 60 : loss 0.454364777 ; accuracy 0.718666673
Step 70 : loss 0.469210893 ; accuracy 0.741571426
Step 80 : loss 0.371560067 ; accuracy 0.75825
Step 90 : loss 0.318025708 ; accuracy 0.774111092
Step 100 : loss 0.334199548 ; accuracy 0.7854
Step 110 : loss 0.372756034 ; accuracy 0.796454549
Step 120 : loss 0.372888565 ; accuracy 0.80491668
Step 130 : loss 0.406678617 ; accuracy 0.813461542
Step 140 : loss 0.463483393 ; accuracy 0.820214272
Step 150 : loss 0.37113446 ; accuracy 0.82586664
Step 160 : loss 0.288563937 ; accuracy 0.8301875
Step 170 : loss 0.222361416 ; accuracy 0.836529434
Step 180 : loss 0.198517278 ; accuracy 0.840888917
Step 190 : loss 0.350897193 ; accuracy 0.844947398
Step 200 : loss 0.189548135 ; accuracy 0.84905
Final step tf.Tensor(200, shape=(), d

## Batching

In real applications batching is essential for performance. The best code to convert to AutoGraph is code where the control flow is decided at the _batch_ level. If making decisions at the individual _example_ level, try to use batch APIs to maintain performance.

For example, if you have the following code in Python:


In [19]:
def square_if_positive(x):
  return [i ** 2 if i > 0 else i for i in x]


square_if_positive(range(-5, 5))

[-5, -4, -3, -2, -1, 0, 1, 4, 9, 16]

You may be tempted to write it in TensorFlow as such (and this would work!):


In [20]:
@tf.function
def square_if_positive_naive(x):
  result = tf.TensorArray(tf.int32, size=x.shape[0])
  for i in tf.range(x.shape[0]):
    if x[i] > 0:
      result = result.write(i, x[i] ** 2)
    else:
      result = result.write(i, x[i])
  return result.stack()


square_if_positive_naive(tf.range(-5, 5))

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([-5, -4, -3, -2, -1,  0,  1,  4,  9, 16])>

But in this case, it turns out you can write the following:


In [21]:
def square_if_positive_vectorized(x):
  return tf.where(x > 0, x ** 2, x)


square_if_positive_vectorized(tf.range(-5, 5))

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([-5, -4, -3, -2, -1,  0,  1,  4,  9, 16])>

In [22]:
tf.function(
    func=None,
    input_signature=None,
    autograph=True,
    experimental_autograph_options=None,
    experimental_relax_shapes=False
)

<function tensorflow.python.eager.def_function.function.<locals>.decorated(inner_function)>

function constructs a callable that executes a TensorFlow graph (tf.Graph) created by tracing the TensorFlow operations in func. This allows the TensorFlow runtime to apply optimizations and exploit parallelism in the computation defined by func.

In [23]:
def f(x, y):
  return tf.reduce_mean(tf.multiply(x ** 2, 3) + y)

g = tf.function(f)

x = tf.constant([[2.0, 3.0]])
y = tf.constant([[3.0, -2.0]])

# `f` and `g` will return the same value, but `g` will be executed as a
# TensorFlow graph.
assert f(x, y).numpy() == g(x, y).numpy()

# Tensors and tf.Variables used by the Python function are captured in the
# graph.
@tf.function
def h():
  return f(x, y)

assert (h().numpy() == f(x, y).numpy()).all()

# Data-dependent control flow is also captured in the graph. Supported
# control flow statements include `if`, `for`, `while`, `break`, `continue`,
# `return`.
@tf.function
def g(x):
  if tf.reduce_sum(x) > 0:
    return x * x
  else:
    return -x // 2

# print and TensorFlow side effects are supported, but exercise caution when
# using Python side effects like mutating objects, saving to files, etc.
l = []

@tf.function
def g(x):
  for i in x:
    print(i)                              # Works
    tf.compat.v1.assign(v, i)                       # Works
    tf.compat.v1.py_func(lambda i: l.append(i))(i)  # Works
    l.append(i)                           # Caution! Doesn't work.

Note that unlike other TensorFlow operations, we don't convert python numerical inputs to tensors. Moreover, a new graph is generated for each distinct python numerical value, for example calling g(2) and g(3) will generate two new graphs (while only one is generated if you call g(tf.constant(2)) and g(tf.constant(3))). Therefore, python numerical inputs should be restricted to arguments that will have few distinct values, such as hyperparameters like the number of layers in a neural network. This allows TensorFlow to optimize each variant of the neural network.

Referencing tf.Variables

The Python function func may reference stateful objects (such as tf.Variable). These are captured as implicit inputs to the callable returned by function. For example:

In [24]:
c = tf.Variable(0)

@tf.function
def f(x):
  c.assign_add(1)
  return x + tf.compat.v1.to_float(c)

assert int(c) == 0
assert f(1.0) == 2.0
assert int(c) == 1
assert f(1.0) == 3.0
assert int(c) == 2

Instructions for updating:
Use `tf.cast` instead.


function can be applied to methods of an object. For example:

In [25]:
class Dense(object):
  def __init__(self):
    self.W = tf.Variable(tf.compat.v1.glorot_uniform_initializer()((10, 10)))
    self.b = tf.Variable(tf.zeros(10))

  @tf.function
  def compute(self, x):
    return tf.matmul(x, self.W) + self.b

d1 = Dense()
d2 = Dense()
x = tf.random.uniform((10, 10))
# d1 and d2 are using distinct variables
assert not (d1.compute(x).numpy() == d2.compute(x).numpy()).all()

Usage with tf.keras

The call methods of a tf.keras.Model subclass can be decorated with function in order to apply graph execution optimizations on it. For example:

In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, keep_probability=0.2):
    super(MyModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(4)
    self.dense2 = tf.keras.layers.Dense(5)
    self.keep_probability = keep_probability

  @tf.function
  def call(self, inputs, training=True):
    y = self.dense2(self.dense1(inputs))
    if training:
      return tf.nn.dropout(y, self.keep_probability)
    else:
      return y

model = MyModel()
model(x, training=True)  # executes a graph, with dropout
model(x, training=False) # executes a graph, without dropout

<tf.Tensor: shape=(10, 5), dtype=float32, numpy=
array([[ 0.5005193 , -0.24730033, -1.0781333 , -0.43008655,  0.1037207 ],
       [ 0.40461332, -0.30323812, -1.645912  , -0.6056957 ,  0.7527479 ],
       [ 0.1423735 , -0.17650397, -1.3146888 , -0.3181464 , -0.25951898],
       [ 0.12431064, -0.22373265, -1.0686699 , -0.28060383,  0.15386477],
       [ 0.8766522 , -0.37231362, -1.0988556 , -0.62091374,  0.63209933],
       [ 0.93118316, -0.37966034, -0.8750252 , -0.6006296 ,  0.7802803 ],
       [-0.18786383,  0.09613264, -0.88458526, -0.19655176,  0.03370802],
       [ 0.78455985, -0.37444085, -0.9614432 , -0.53267145,  0.54869545],
       [ 0.4642647 , -0.2502508 , -1.1140661 , -0.4421596 ,  0.25081718],
       [ 0.4145861 , -0.54091966, -1.2519972 , -0.33458346,  0.10882222]],
      dtype=float32)>

Input Signatures

function instantiates a separate graph for every unique set of input shapes and datatypes. For example, the following code snippet will result in three distinct graphs being traced, as each input has a different shape.

In [27]:
@tf.function
def f(x): return tf.add(x, 1.)

scalar = tf.constant(1.0)
vector = tf.constant([1.0, 1.0])
matrix = tf.constant([[3.0]])

f(scalar)
f(vector)
f(matrix)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[4.]], dtype=float32)>

An "input signature" can be optionally provided to function to control the graphs traced. The input signature specifies the shape and type of each Tensor argument to the function using a tf.TensorSpec object. For example, the following code snippet ensures that a single graph is created where the input Tensor is required to be a floating point tensor with no restrictions on shape.

In [28]:
@tf.function(input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])
def f(x): return tf.add(x, 1.)

When an input_signature is specified, the callable will convert the inputs to the specified TensorSpecs.

Tracing and staging

When autograph is True, all Python control flow that depends on Tensor values is staged into a TensorFlow graph. When autograph is False, the function is traced and control flow is not allowed to depend on data.

Note that function only stages TensorFlow operations, all Python code that func executes and does not depend on data will shape the construction of the graph. For example, consider the following:

In [29]:
import numpy as np

def add_noise():
  return tf.eye(5) + np.random.randn(5, 5)

traced = tf.function(add_noise)

add_noise() will return a different output every time it is invoked. However, traced() will return the same value every time it is called, since a particular random value generated by the np.random.randn call will be inserted in the traced/staged TensorFlow graph as a constant. In this particular example, replacing np.random.randn(5, 5) with tf.random.normal((5, 5)) will result in the same behavior for add_noise() and traced().

Python Side-Effects

A corollary of the previous discussion on tracing is the following: If a Python function func has Python side-effects, then executing func multiple times may not be semantically equivalent to executing F = tf.function(func) multiple times; this difference is due to the fact that function only captures the subgraph of TensorFlow operations that is constructed when func is invoked to trace a graph.

The same is true if code with Python side effects is used inside control flow, such as a loop. If your code uses side effects that are not intended to control graph construction, wrap them inside tf.compat.v1.py_func.

Retracing

A single tf.function object might need to map to multiple computation graphs under the hood. This should be visible only as performance (tracing graphs has a nonzero computational and memory cost) but should not affect the correctness of the program. A traced function should return the same result as it would when run eagerly, assuming no unintended Python side-effects.

Calling a tf.function with tensor arguments of different dtypes should lead to at least one computational graph per distinct set of dtypes. Alternatively, always calling a tf.function with tensor arguments of the same shapes and dtypes and the same non-tensor arguments should not lead to additional retracings of your function.

Other than that, TensorFlow reserves the right to retrace functions as many times as needed, to ensure that traced functions behave as they would when run eagerly and to provide the best end-to-end performance. For example, the behavior of how many traces TensorFlow will do when the function is repeatedly called with different python scalars as arguments is left undefined to allow for future optimizations.

To control the tracing behavior, use the following tools: - different tf.function objects are guaranteed to not share traces; and - specifying a signature or using concrete function objects returned from get_concrete_function() guarantees that only one function graph will be built.

Args:
func: function to be compiled. If func is None, returns a decorator that can be invoked with a single argument - func. The end result is equivalent to providing all the arguments up front. In other words, tf.function(input_signature=...)(func) is equivalent to tf.function(func, input_signature=...). The former can be used to decorate Python functions, for example: @tf.function(input_signature=...) def foo(...): ...
input_signature: A possibly nested sequence of tf.TensorSpec objects specifying the shapes and dtypes of the Tensors that will be supplied to this function. If None, a separate function is instantiated for each inferred input signature. If input_signature is specified, every input to func must be a Tensor, and func cannot accept **kwargs.
autograph: Whether autograph should be applied on func before tracing a graph. This allows for dynamic control flow (Python if's, loops etc.) in the traced graph. See https://www.tensorflow.org/guide/autograph for more information.
experimental_autograph_options: Experimental knobs (in the form of a tuple of tensorflow.autograph.Feature values) to control behavior when autograph=True.
experimental_relax_shapes: When true, argument shapes may be relaxed to avoid unecessary retracing.
Returns:
If func is not None, returns a callable that will execute the compiled function (and return zero or more tf.Tensor objects). If func is None, returns a decorator that, when invoked with a single func argument, returns a callable equivalent to the case above.

Raises:
TypeError: If input_signature is neither None nor a sequence of TensorSpec objects.